In [360]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [361]:
movies = pd.read_csv(Path("archive/movies.csv"))

In [362]:
print(movies.shape)

(58098, 3)


In [363]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [364]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [365]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [366]:
movies["year_of_release"] = 0

In [367]:
movies.head()

,movieId,title,genres,year_of_release
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0
4,5,Father of the Bride Part II (1995),Comedy,0


In [368]:
# seperating year of release from title
for idx in movies.index:
    title = movies.loc[idx,"title"]
    year_of_release = re.findall(r"\(([0-9]+)\)", title)
    for year in year_of_release:
        if year_of_release is None:
            movies.loc[idx,"year_of_release"] = np.NaN
        else:
            movies.loc[idx,"year_of_release"] = year


In [369]:
movies.isnull().sum()

movieId            0
title              0
genres             0
year_of_release    0
dtype: int64

In [370]:
movies.loc[movies["year_of_release"].isnull()]

,movieId,title,genres,year_of_release


In [371]:
movies.loc[movies.title == "Trails (Veredas) (1978)"]

,movieId,title,genres,year_of_release
17343,87061,Trails (Veredas) (1978),(no genres listed),1978


In [372]:
movies_new = pd.DataFrame(columns=["movieId","title","genre","year_of_release"])
movies_new.head()

,movieId,title,genre,year_of_release


In [373]:
for idx in movies.index:
    genre_list = movies.loc[idx,"genres"].split("|")
    for genre in genre_list:
        temp_df = pd.DataFrame(
            [[movies.loc[idx,"movieId"], movies.loc[idx,"title"],genre,movies.loc[idx,"year_of_release"]]],
            columns=["movieId","title","genre","year_of_release"]
            )
        movies_new = pd.concat([movies_new, temp_df], ignore_index= True)

In [374]:
movies_new.head()

,movieId,title,genre,year_of_release
0,1,Toy Story (1995),Adventure,1995
1,1,Toy Story (1995),Animation,1995
2,1,Toy Story (1995),Children,1995
3,1,Toy Story (1995),Comedy,1995
4,1,Toy Story (1995),Fantasy,1995


In [375]:
movies_new.head()

,movieId,title,genre,year_of_release
0,1,Toy Story (1995),Adventure,1995
1,1,Toy Story (1995),Animation,1995
2,1,Toy Story (1995),Children,1995
3,1,Toy Story (1995),Comedy,1995
4,1,Toy Story (1995),Fantasy,1995


In [376]:
genre_set = set(movies_new["genre"].values)

In [377]:
genres = pd.DataFrame(columns=["genreId", "genre"])
i = 0
for genre in genre_set:
    temp_df = pd.DataFrame([[i, genre]], columns=["genreId", "genre"])
    genres = pd.concat([genres, temp_df],ignore_index = True)
    i += 1

In [378]:
genres 

,genreId,genre
0,0,Adventure
1,1,Action
2,2,IMAX
3,3,War
4,4,Film-Noir
5,5,Western
6,6,Documentary
7,7,Animation
8,8,Drama
9,9,Horror


In [379]:
movie_genre_join_table = pd.DataFrame(columns=["movieId","genreId"])
movie_genre_join_table

,movieId,genreId


In [380]:
dict_genre = {'Adventure':0,'Action':1,	'IMAX':2, 'War':3,'Film-Noir':4,'Western':5,'Documentary':6,'Animation':7,'Drama':8, 'Horror':9,'Mystery':10,
'Comedy':11,'Children':12,'Musical':13,'Romance':14,'Fantasy':15,'Sci-Fi':16,'Thriller':17,'(no genres listed)':18,'Crime':19}

In [381]:
for idx in movies_new.index:
    tag_id = dict_genre[movies_new.loc[idx,'genre']]
    movie_id = movies_new.loc[idx,"movieId"]
    temp = pd.DataFrame([[movie_id, tag_id]], columns=["movieId","genreId"])
    movie_genre_join_table = pd.concat([movie_genre_join_table, temp])

In [382]:
movie_genre_join_table.head()

,movieId,genreId
0,1,0
0,1,7
0,1,12
0,1,11
0,1,15


In [383]:
movies_new.drop(columns=["genre"],inplace=True)

In [384]:
movies_new.head()

,movieId,title,year_of_release
0,1,Toy Story (1995),1995
1,1,Toy Story (1995),1995
2,1,Toy Story (1995),1995
3,1,Toy Story (1995),1995
4,1,Toy Story (1995),1995


In [385]:
movies_new = movies_new.drop_duplicates()
movies_new.head()

,movieId,title,year_of_release
0,1,Toy Story (1995),1995
5,2,Jumanji (1995),1995
8,3,Grumpier Old Men (1995),1995
10,4,Waiting to Exhale (1995),1995
13,5,Father of the Bride Part II (1995),1995


In [386]:
movies_new = movies_new.reset_index()

In [387]:
movies_new.drop(columns=["index"],inplace = True)

In [388]:
movies_new.head()

,movieId,title,year_of_release
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [389]:
movie_genre_join_table = movie_genre_join_table.reset_index()

In [390]:
movie_genre_join_table.drop(columns=["index"],inplace = True)

In [391]:
movie_genre_join_table.tail()

,movieId,genreId
106102,193882,9
106103,193882,16
106104,193886,1
106105,193886,19
106106,193886,8


In [392]:
genres.head()

,genreId,genre
0,0,Adventure
1,1,Action
2,2,IMAX
3,3,War
4,4,Film-Noir


In [393]:
movies_new.head()

,movieId,title,year_of_release
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [394]:
tags = pd.read_csv(Path("archive/tags.csv"))

In [395]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [396]:
movie_genre_join_table["genreId"] == 18

0         False
1         False
2         False
3         False
4         False
          ...  
106102    False
106103    False
106104    False
106105    False
106106    False
Name: genreId, Length: 106107, dtype: bool

In [397]:
empty_genre_list = movie_genre_join_table.loc[movie_genre_join_table["genreId"] == 18]

In [398]:
empty_genre_list.head()

,movieId,genreId
34699,83773,18
34728,83829,18
35071,84768,18
35672,86493,18
35973,87061,18


In [399]:
movie_genre_join_table.to_csv(Path("preproc_datasets/movie_genre_join.csv"))

In [400]:
movies_new.to_csv(Path("preproc_datasets/movies.csv"))

In [401]:
genres.to_csv(Path("preproc_datasets/genres.csv"))